In [23]:
import json

data_path = '/Users/myding/Desktop/questions_v16_3_4/'
des_path = data_path + 'open_end_questions.json'
mc_path = data_path + 'multiple_choice_questions.json'


answer_list = []

question_data = json.load(open(des_path))

for index, questions in enumerate(question_data):
    if (index + 1) % 100 == 0:
        print(index)

    for item in questions["questions"]:
        data_item = {}
        question = item['question']
        answer = item['answer']
        words = question.replace('?', '').strip().split()
        data_item['answer'] = answer
        data_item['question'] = question
        data_item['video_id'] = 'video' + str(index)
        data_item['answer_type'] = item['question_family']
        print(json.dumps(data_item))
        word_set = answer.split()
        word_set.sort()
        answer_list.append('_'.join(word_set))

{"answer": "yes", "question": "Are there any light moving gray objects when the video ends?", "video_id": "video0", "answer_type": "object_mass_exist"}


In [19]:
question_data = json.load(open(mc_path))

for index, questions in enumerate(question_data):
    if (index + 1) % 100 == 0:
        print(index)
    for item in questions["questions"]:
        # print(item)
        # break
        question = item['question']
        correct = item['correct']
        answer = 'True'
        answer_list.append(answer)
        for i in correct:
            data_item = {}
            data_item['question'] = question + ' ' + i[0]
            data_item['answer'] = answer
            data_item['video_id'] = 'video' + str(index)
            data_item['answer_type'] = item['question_type']
        wrong = item['wrong']
        answer = 'False'
        answer_list.append(answer)
        for i in wrong:
            data_item = {}
            data_item['question'] = question + ' ' + i[0]
            data_item['answer'] = answer
            data_item['video_id'] = 'video' + str(index)
            data_item['answer_type'] = item['question_type']

99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599
1699
1799
1899
1999
2099
2199
2299
2399
2499
2599
2699
2799
2899
2999
3099
3199
3299
3399
3499
3599
3699
3799
3899
3999
4099
4199
4299
4399
4499
4599
4699
4799
4899
4999
5099
5199
5299
5399
5499
5599
5699
5799
5899
5999
6099
6199
6299
6399
6499
6599
6699
6799
6899
6999
7099
7199
7299
7399
7499
7599
7699
7799
7899
7999
8099
8199
8299
8399
8499
8599
8699
8799
8899
8999
9099
9199
9299
9399
9499
9599
9699
9799
9899
9999


In [20]:
answer_dict = {}
answer_vocab = list(set(answer_list))
for index, item in enumerate(answer_vocab):
    answer_dict[item] = index

In [21]:
answer_dict

{'left': 0,
 'green': 1,
 'and_gray_yellow': 2,
 'gray': 3,
 'and_purple_red': 4,
 'and_blue_yellow': 5,
 'and_blue_red': 6,
 'and_blue_purple': 7,
 'and_cylinder_sphere': 8,
 'and_gray_green': 9,
 '2': 10,
 'cube': 11,
 'and_gray_purple': 12,
 'yes': 13,
 'and_cube_sphere': 14,
 'rubber': 15,
 'and_cyan_red': 16,
 'and_cyan_purple': 17,
 'no': 18,
 'and_cyan_yellow': 19,
 'and_cyan_green': 20,
 'True': 21,
 'right': 22,
 'and_gray_red': 23,
 'and_blue_green': 24,
 'and_brown_green': 25,
 'cylinder': 26,
 'and_brown_red': 27,
 'and_blue_gray': 28,
 'sphere': 29,
 'down': 30,
 'and_cube_cylinder': 31,
 '4': 32,
 'and_blue_cyan': 33,
 'and_cyan_gray': 34,
 'and_green_yellow': 35,
 'and_green_purple': 36,
 'and_brown_gray': 37,
 'False': 38,
 '3': 39,
 'cyan': 40,
 'yellow': 41,
 'and_blue_brown': 42,
 'and_brown_purple': 43,
 '0': 44,
 'and_brown_cyan': 45,
 'red': 46,
 'and_purple_yellow': 47,
 'and_green_red': 48,
 'metal': 49,
 'brown': 50,
 '1': 51,
 'up': 52,
 'purple': 53,
 'and_br

In [ ]:
data_dir = '/Users/myding/Desktop/demo25/frames'

ffmpeg -framerate 25 -loop 0 -f image2 -i /Users/myding/Desktop/demo25/frames/frame_%05d.png \
                            -pix_fmt yuv420p -vcodec libx264 -crf 23 data.mp4

In [1]:
import lmdb
import cv2
import os
import multiprocessing as mp
from tqdm import tqdm
from os.path import join


def read_raw_binary_file(file_path):
    """can actually be any file"""
    with open(file_path, 'rb') as f:
        return f.read()


def read_raw_binary_from_pair(id_path_pair):
    file_id, filepath = id_path_pair
    try:
        file_byte_str = read_raw_binary_file(filepath)
        return file_id, file_byte_str
    except Exception as e:
        return filepath, None


def get_abspaths_by_ext(dir_path, ext=(".jpg",)):
    """Get absolute paths to files in dir_path with extensions specified by ext.
    Note this function does work recursively.
    """
    if not isinstance(ext, tuple):
        ext = tuple([ext, ])
    filepaths = [os.path.join(root, name)
                 for root, dirs, files in os.walk(dir_path)
                 for name in files
                 if name.endswith(tuple(ext))]
    return filepaths


def read_compress_raw_img(img_path):
    """
    Args:
        img_path: str, path to a raw image (.jpg, ....)
    """
    img = cv2.imread(img_path)  # np array, uint8
    encoded_img_arr = cv2.imencode('.jpg', img)[1]
    return encoded_img_arr


def read_raw_img_from_pair(id_path_pair):
    img_id, img_path = id_path_pair
    try:
        encoded_img_arr = read_compress_raw_img(img_path)
        return img_id, encoded_img_arr
    except Exception as e:
        return img_path, None


def write_lmdb_from_id_path(
        id_path_pairs, lmdb_save_dir, num_workers,
        lmdb_preprocessing_fn=read_raw_img_from_pair):
    """
    Write
    Args:
        id_path_pairs: list(tuple), each tuple is (file_id, filepath)
        lmdb_save_dir: str, path to save lmdb
        num_workers: int
        lmdb_preprocessing_fn: a function that takes two args [file_id (str), filepath (str)]
            and returns [file_id (byte string), value (byte string)] that will be used as
            key and value for lmdb txn.put(key=file_id, value=value).
            Additionally, this function should handle exceptions, where it should return key as
            the filepath, and return value as None.

    """
    env = lmdb.open(lmdb_save_dir, map_size=1024**4)
    txn = env.begin(write=True)
    error_filepaths = []
    if num_workers > 1:
        with mp.Pool(num_workers) as pool, tqdm(total=len(id_path_pairs)) as pbar:
            for idx, (key, value) in enumerate(
                    pool.imap_unordered(
                        lmdb_preprocessing_fn, id_path_pairs, chunksize=128)):
                if value is None:
                    error_filepaths.append(key)
                    continue
                txn.put(key=str(key).encode("utf-8"), value=value)
                if idx % 1000 == 0:
                    txn.commit()
                    txn = env.begin(write=True)
                pbar.update(1)
    else:
        for idx, pair in tqdm(enumerate(id_path_pairs), total=len(id_path_pairs)):
            key, value = lmdb_preprocessing_fn(pair)
            if value is None:
                error_filepaths.append(key)
                continue
            txn.put(key=str(key).encode("utf-8"), value=value)
            if idx % 1000 == 0:
                txn.commit()
                txn = env.begin(write=True)

    if len(error_filepaths) > 0:
        with open(join(lmdb_save_dir, "error_filepaths.log"), "w") as f:
            f.write("\n".join(error_filepaths))
        print(f"There are {len(error_filepaths)} files raised exceptions, "
              f"3 examples are {error_filepaths[:3]}")
    txn.commit()
    env.close()

In [4]:
import os


def get_abspaths_by_ext(dir_path, ext=(".jpg",)):
    """Get absolute paths to files in dir_path with extensions specified by ext.
    Note this function does work recursively.
    """
    if not isinstance(ext, tuple):
        ext = tuple([ext, ])
    filepaths = [os.path.join(root, name)
                 for root, dirs, files in os.walk(dir_path)
                 for name in files
                 if name.endswith(tuple(ext))]
    return filepaths


def get_filename(filepath):
    return os.path.splitext(os.path.split(filepath)[1])[0]


def main_convert():
    lmdb_save_dir = 'output/'
    file_type='video'
    if os.path.exists(lmdb_save_dir) and os.listdir(lmdb_save_dir):
        raise ValueError(f"lmdb_save_dir {lmdb_save_dir} already exists and is not empty")
    else:
        os.makedirs(lmdb_save_dir, exist_ok=True)

    file_paths = get_abspaths_by_ext(dir_path='/Users/myding/Desktop/demo25/frames/', ext=('mp4',))
    id_path_pairs = [(get_filename(p), p) for p in file_paths]
    lmdb_preprocessing_fn = read_raw_binary_from_pair \
        if file_type == "video" else read_raw_img_from_pair
    write_lmdb_from_id_path(
        id_path_pairs=id_path_pairs,
        lmdb_save_dir=lmdb_save_dir,
        lmdb_preprocessing_fn=lmdb_preprocessing_fn,
        num_workers=4
    )

In [5]:
main_convert()

100%|██████████| 1/1 [00:00<00:00, 232.46it/s]


In [6]:
import io

env = lmdb.open('output/', readonly=True, create=False)
txn = env.begin(buffers=True)
io_stream = io.BytesIO(txn.get(str('data').encode("utf-8")))

In [19]:
for key, value in txn.cursor():
  print (io.BytesIO(key).getvalue(), io.BytesIO(value))

b'data' <memory at 0x101289f2288>


In [21]:
import sys
import lmdb
import cv2
sys.path.insert(0, '.')
from src.datasets.dataset_base import extract_frames_from_video_binary

import io

env = lmdb.open('output/', readonly=True, create=False)
txn = env.begin(buffers=True)
io_stream = io.BytesIO(txn.get(str('data').encode("utf-8")))

for key, value in txn.cursor():
    key = io.BytesIO(key).getvalue().decode('utf-8')
    value = io.BytesIO(value)
    frames = extract_frames_from_video_binary(value)[0]
    print(frames[0].transpose(0, 2).shape)
    for i in range(3):
        cv2.imwrite(f'{i}.png', frames[i].transpose(0, 2).numpy())

torch.Size([480, 320, 3])
